In [1]:
!nvidia-smi

Fri May 26 14:07:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   32C    P0    41W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# sync python module
%load_ext autoreload
%autoreload 2

In [5]:
!pwd

/nfs/Workspace/CardiacSeg/exps/exps/swinunetr/chgh


In [14]:
import os

workspace_dir = '/nfs/Workspace/CardiacSeg'
model_name = 'cotr' #'unetcnx_a1'
data_name = 'chgh'
sub_data_dir_name = 'dataset_2'
exp_name = 't_4' #'exp_b7_9_ds1_tn1'
data_dict_file_name = 'exp_b7_9.json'

tune_mode = 'test' #'lrschedule' #'train'


root_exp_dir = os.path.join(
    workspace_dir, 
    'exps',
    'exps',
    model_name,
    data_name,
    'tune_results'
)

root_data_dir = os.path.join(
    workspace_dir, 
    'dataset',
    data_name
)

data_dir = os.path.join(root_data_dir, sub_data_dir_name)
data_dicts_json = os.path.join(workspace_dir, 'exps', 'data_dicts', data_name, data_dict_file_name)

# unetcnx_a1
# exp_dir = '/nfs/Workspace/CardiacSeg/exps/exps/unetcnx_a1/chgh/tune_results/exp_b7_9_ds1_tn1/main_5cbaa_00000_0_optim=lr_0_0007_weight_decay_0_0005_2023-05-17_05-19-13/'
# unet3d
# exp_dir = '/nfs/Workspace/CardiacSeg/exps/exps/unet3d/chgh/tune_results/t_4/main_4227b_00000_0_exp=exp_t_4_2023-04-02_15-44-31'
# attn unet
# exp_dir = '/nfs/Workspace/CardiacSeg/exps/exps/attention_unet/chgh/tune_results/t_4/main_88147_00000_0_exp=exp_t_4_2023-04-02_12-11-43'
# cotr
# exp_dir = '/nfs/Workspace/CardiacSeg/exps/exps/cotr/chgh/tune_results/t_4/main_0ef8c_00000_0_exp=exp_t_4_2023-04-02_10-06-38'
# unetr
# exp_dir = '/nfs/Workspace/CardiacSeg/exps/exps/unetr/chgh/tune_results/t_4/main_4c480_00000_0_exp=exp_t_4_2023-04-02_06-47-55'
# swinunetr
# exp_dir = '/nfs/Workspace/CardiacSeg/exps/exps/swinunetr/chgh/tune_results/t_4/main_634c8_00000_0_exp=exp_exp_b7_9_2023-03-27_02-09-21'

model_dir = os.path.join(exp_dir, 'models')
log_dir = os.path.join(exp_dir, 'logs')
eval_dir = os.path.join(exp_dir, 'eda_evals')

best_checkpoint = os.path.join(model_dir, 'best_model.pth')
final_checkpoint = os.path.join(model_dir, 'final_model.pth')

# test train data or val data for eda (train, val), default is test data
# val dice value diff training, because infer not include bg
eda_test_data = 'test' # val, test

os.makedirs(root_exp_dir, exist_ok=True)

%cd {root_exp_dir}/../

/nfs/Workspace/CardiacSeg/exps/exps/cotr/chgh


In [15]:
!PYTHONPATH=/nfs/Workspace/CardiacSeg /opt/conda/bin/python /nfs/Workspace/CardiacSeg/expers/train.py \
--tune_mode={tune_mode} \
--exp_name={exp_name} \
--data_name={data_name} \
--data_dir={data_dir} \
--root_exp_dir={root_exp_dir} \
--model_name={model_name}\
--model_dir={model_dir} \
--log_dir={log_dir} \
--eval_dir={eval_dir} \
--start_epoch=0 \
--val_every=20 \
--max_early_stop_count=20 \
--max_epoch=8000  \
--data_dicts_json={data_dicts_json} \
--pin_memory \
--out_channels=2 \
--patch_size=4 \
--feature_size=48 \
--drop_rate=0.1 \
--depths 3 3 9 3 \
--kernel_size 7 \
--exp_rate 4 \
--norm_name='layer' \
--a_min=-42 \
--a_max=423 \
--space_x=0.7 \
--space_y=0.7 \
--space_z=1.0 \
--roi_x=128 \
--roi_y=128 \
--roi_z=128 \
--optim='AdamW' \
--lr=5e-4 \
--weight_decay=5e-4 \
--checkpoint={best_checkpoint} \
--use_init_weights \
--deep_sup \
--eda_test_data=eda_test_data \
--infer_post_process \
--test_mode

cuda is available
model: cotr
nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  + Number of Backbone Params: 20.50(e6)
  + Number of Transformer Params: 9.32(e6)
optimzer: AdamW
{'lr': 0.0005, 'weight_decay': 0.0005}
=> loaded checkpoint '/nfs/Workspace/CardiacSeg/exps/exps/cotr/chgh/tune_results/t_4/main_0ef8c_00000_0_exp=exp_t_4_2023-04-02_10-06-38/models/best_model.pth' (epoch 301) (bestacc 0.8893863558769226) (early stop count 0)
load json from /nfs/Workspace/CardiacSeg/exps/data_dicts/chgh/exp_b7_9.json
train files (9): ['pid_08', 'pid_27', 'pid_30', 'pid_52', 'pid_56', 'pid_57', 'pid_110', 'pid_1002', 'pid_1003']
val files (3): ['pid_08_1', 'pid_107', 'pid_108']
test files (3): ['pid_02', 'pid_106', 'pid_1000']
infer data: {'image': '/nfs/Workspace/CardiacSeg/dataset/chgh/dataset_2/pid_02/pid_02.nii.gz', 'label': '/nfs/Workspace/CardiacSeg/dataset/chgh/dataset_2/pid_02/pid_02_gt.nii.gz'}
<class 'monai.transforms.utility.array.AddChannel'>: Class `Add